In [ ]:
import sys
sys.path.insert(0, r'D:\Github\zi_labrad')

%matplotlib inline
import os
from time import sleep
from importlib import reload
import logging
import matplotlib.pyplot as plt
import numpy as np
import qcodes as qc


logging.basicConfig(format='%(asctime)s | %(name)s [%(levelname)s] : %(message)s',
                    level=logging.INFO
                    )

In [ ]:
import labrad
from labrad.units import Unit,Value
_unitSpace = ('V','mV','us','ns','s','GHz','MHz','kHz','Hz','dBm','rad','None')
V, mV, us, ns,s, GHz, MHz,kHz,Hz, dBm, rad,_l = [Unit(s) for s in _unitSpace]

from zilabrad import *
from zilabrad.pyle.util import sweeptools
ar = sweeptools.RangeCreator()

In [ ]:
_default_modes = [1,2,3,4,5]
devices = [None]*len(_default_modes)

cxn=labrad.connect()
ss = switchSession(cxn,user='hwh',session=None)

In [ ]:
from zilabrad.instrument.qubitServer import loadQubits

In [ ]:
sample, qubits = loadQubits(ss, write_access=False)

In [ ]:
Registry = qc.config
Reg = Registry.update_config('M:\Registry\qcodesrc.json')
# Reg now is only a wrapped dict

reg = Reg['user']
# reg give the parameters that we set

In [ ]:
def checkID():
    if 'devicesID' not in reg.keys():
        devicesid = {
         'microwave_source_readout': '192.168.1.241',
         'microwave_source_xy': '192.168.1.240',
         'yokogawa_dc_ip': '192.168.1.239',
         'zi_hd_id': 'dev8334',
         'zi_qa_id': 'dev2591'}
        Registry.add('devicesID',devicesid)
        Registry.save_to_home()
    else:
        devicesid = reg['devicesID']
        
    logging.info(devicesid)

In [ ]:
checkID()
reg['devicesID']

In [ ]:
def bringup_device(modes,reg):
    zH = zurichHelper
    dev = reg['devicesID']
    for m in modes: 
        if m == 1:
            qa = zH.zurich_qa(dev['zi_qa_id'])
            devices[m-1] = qa
        if m == 2:
            hd = zH.zurich_hd(dev['zi_hd_id'])
            devices[m-1] = hd
        if m == 3:
            mw = zH.microwave_source(dev['microwave_source_xy'],'mw')
            devices[m-1] = mw
        if m == 4:
            mw_r = zH.microwave_source(dev['microwave_source_readout'],'mw_r')
            devices[m-1] = mw_r
        if m == 5:
            wfs = waveforms.waveform()
            devices[m-1] = wfs
    return 

In [ ]:
# bringup all
bringup_device(_default_modes,reg)

In [ ]:
# bringup_device([3],reg)

In [ ]:
def RunAllDummy(*args):
    exp_devices = [None]*4
    RunAllExperiment(exp_devices,*args)
    return

RunAllDummy = mp.RunAllExp

In [ ]:
def runDummy(qubits,exp_devices):
    for q in qubits: ## 多比特
        if 'do_readout' in q.keys():
            if 'r' not in q.keys():
                q.demod_phase = q.qa_adjusted_phase[Hz]*(qa.adc_trig_delay_s) ## adjusted phase
                q.r = waveforms.readout(amp=q.power_r,phase=q.demod_phase,freq=q.demod_freq,start=0,length=q.readout_len)
    return 
    
mp.runQ = runDummy

In [ ]:
reload(mp)
mp.exp_devices = devices
mp.exp_devices

In [ ]:
# mp.s21_scan(ss,freq=ar[6.0:6.1:0.01,GHz])

In [ ]:
import time
time0=time.time()

In [ ]:
from math import ceil,pi

In [ ]:
(1*us)['s']

In [ ]:
print('array',np.arange(100))

In [ ]:
# 100 us

all_length= 1e-6
fs=1.8e9
sampling_num = ceil(all_length*fs/16)*16

In [ ]:
sampling_num

In [ ]:
funcCos = lambda t: np.cos(6*t)

def toArray_Array(points):
    tlist = np.arange(points)
    funcArray = [funcCos(t) for t in tlist]
    return

def toArray2_Generator(points):
    tlist = range(points)
    funcArray = [funcCos(t) for t in tlist]
    return

In [ ]:
sampling_num = 1808

In [ ]:

time0=time.time()

arrays = toArray_Array(points = sampling_num)
print(time.time()-time0)

In [ ]:
np.arange(0.1,0.9,0.5)

In [ ]:
from math import ceil
import time
import numpy as np

In [ ]:
from zilabrad.instrument.waveforms import cosine

In [ ]:
class waveform(object):
    def __init__(self,all_length=1e-6,fs=1.8e9,origin=0):
        self.fs = fs
        self.sample_number = ceil(all_length*self.fs/16)*16 ## QA lenght最小16,最小单位间隔8; HD length最小32,最小单位间隔16;
        self.tlist = []
    def func2array(self,envelopes,start=None,end=None,fs=None):
        if start == None:
            start = envelopes.start
        if end == None:
            end = envelopes.end
            
        if len(self.tlist) == 0:
            if fs == None:
                fs = self.fs
            tlist = np.arange(start,end,1/fs)
        else:
            tlist = self.tlist
        
        pulse = [envelopes(t) for t in tlist]
        return pulse
    def set_tlist(self,origin,end,fs):
        self.tlist = np.arange(origin,end,1./fs)

In [ ]:
class waveform_gene(object):
    def __init__(self,all_length=1e-6,fs=1.8e9,origin=0):
        self.fs = fs
        self.sample_number = ceil(all_length*self.fs/16)*16 ## QA lenght最小16,最小单位间隔8; HD length最小32,最小单位间隔16;
    
    def func2array(self,envelopes,start=None,end=None,fs=None):
        t0 = time.time()
        if fs == None:
            fs = self.fs
        if start == None:
            start = envelopes.start
        if end == None:
            end = envelopes.end
            
        if end <= start:
            return []
        
        interval = 1./self.fs
        steps = ceil((end - start)/interval)
        pulse = [envelopes(x*interval+start) for x in range(steps)]
        return pulse

    def func2array_np(self,npfunc,start=None,end=None,fs=None):
        t0 = time.time()
        if fs == None:
            fs = self.fs
        if start == None:
            start = envelopes.start
        if end == None:
            end = envelopes.end
            
        if end <= start:
            return []
        interval = 1./self.fs
#         steps = ceil((end - start)/interval)
        return npfunc(np.arange(start,end,interval))

In [ ]:
fs=1.8e9
wf_taoziyu3 = waveform_gene(fs)
t0 = time.time()

def cosine3(x):
    return np.cos(x)+np.cos(10*x)
pulses3 = []
for _ in range(80):
    pulses3.append(wf_taoziyu3.func2array_np(cosine3,start=0.,end=10e-6))
print("Peach update2 Cost %.4f ms" % (1e3*(time.time()-t0) ) )